In [49]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import random
import sklearn as sk
from time import time
import pickle

In [88]:
df_anon=pd.read_csv("../data/MIXED50patients_subsamples/1/Baynet/brca_mRNA_patients_BayNet_n20_deg2_eps0_seed_1.csv")

df_ext = pd.read_csv("../data/MIXED50patients_subsamples/1/brca_mRNA_50patients.csv")

In [73]:
df_anon.columns.tolist()




['CANCER_TYPE_DETAILED',
 'ER_STATUS',
 'HER2_STATUS',
 'GRADE',
 'PR_STATUS',
 'TUMOR_SIZE',
 'TUMOR_STAGE',
 'TMB_NONSYNONYMOUS',
 'LYMPH_NODES_EXAMINED_POSITIVE',
 'NPI',
 'CELLULARITY',
 'CHEMOTHERAPY',
 'COHORT',
 'ER_IHC',
 'HER2_SNP6',
 'HORMONE_THERAPY',
 'INFERRED_MENOPAUSAL_STATE',
 'INTCLUST',
 'AGE_AT_DIAGNOSIS',
 'OS_MONTHS',
 'CLAUDIN_SUBTYPE',
 'THREEGENE',
 'VITAL_STATUS',
 'LATERALITY',
 'RADIO_THERAPY',
 'HISTOLOGICAL_SUBTYPE',
 'BREAST_SURGERY',
 'RFS_STATUS',
 'RFS_MONTHS']

In [74]:
matched_nodes = set()
# find nodes in anonymized graph with same occupation as individuals in external dataset
for idx, row in df_ext.iterrows():
    tumor_size = row['TUMOR_SIZE']
    matches = df_anon.index[(df_anon["TUMOR_SIZE"] == tumor_size)].tolist()
    matched_nodes.update(matches)

In [75]:
def calculate_linkage_quality(anon_data, ext_data):
    # find individuals in anonymized data with same occupation as individuals in external data
    matched_rows = set()
    for i in range(len(ext_data)):
        matches = []
        for j in range(len(anon_data)):
            if all(ext_data[i, k] == anon_data[j, k] for k in range(ext_data.shape[1])):
                matches.append(j)
        matched_rows.update(matches)

    # calculate linkage quality
    matched_pairs = []
    for i in matched_rows:
        for j in matched_rows:
            if i != j and (i, j) not in matched_pairs and (j, i) not in matched_pairs:
                if all(anon_data[i, k] == anon_data[j, k] for k in range(anon_data.shape[1])):
                    matched_pairs.append((i, j))
    print(matched_pairs)
    linkage_quality3 = len(matched_pairs) / len(ext_data)

    return linkage_quality3

In [76]:
print(matched_nodes)
# calculate linkage quality
linkage_quality2 = calculate_linkage_quality(df_anon, df_ext)

print("Linkage quality: ", linkage_quality2)

{0, 1, 3, 4, 5, 7, 10, 11, 12, 15, 16, 17, 18, 21, 28, 31, 32, 33, 37, 38, 41, 42, 44, 45, 46, 47}


KeyError: (0, 0)

In [77]:
anon_data2 = np.random.randint(0, 100, size=(40, 100))
ext_data2 = np.random.randint(0, 100, size=(100, 100))

anon_data2

array([[24, 15, 86, ..., 73, 27, 53],
       [33, 75, 19, ..., 25, 42, 79],
       [84, 23, 15, ...,  9,  9, 65],
       ...,
       [25, 18, 41, ...,  5, 62, 95],
       [51, 37, 51, ..., 65, 34, 73],
       [10, 53, 36, ..., 27, 13, 57]])

In [78]:
## Re-identification rate: The percentage of individuals in the anonymized graph that can be re-identified using external information.

##Linkage quality: A measure of how well the linkage between the anonymized graph and the external dataset captures the true relationships between individuals


In [79]:
# linkage_quality = calculate_linkage_quality(anon_data2, ext_data2)
#
# print("Linkage quality: ", linkage_quality)

In [80]:
def calculate_linkage_quality(anon_df, real_df, threshold):
    # identify columns to use for similarity calculation
    similarity_columns = df_anon.columns.tolist()

    # create similarity matrix
    similarity_matrix = np.zeros((len(anon_df), len(real_df)))
    for i in range(len(anon_df)):
        for j in range(len(real_df)):
            similarities = []
            for column in similarity_columns:
                if anon_df.at[i, column] == real_df.at[j, column]:
                    similarities.append(1)
                else:
                    similarities.append(0)
            similarity_matrix[i, j] = sum(similarities) / len(similarity_columns)

    # find matches between anonymized and real data sets based on similarity threshold
    matches = []
    for i in range(len(anon_df)):
        for j in range(len(real_df)):
            if similarity_matrix[i, j] >= threshold:
                matches.append((i, j))

    # calculate linkage quality
    matched_pairs = []
    for pair in matches:
        i, j = pair
        if (i, j) not in matched_pairs and (j, i) not in matched_pairs:
            matched_pairs.append((i, j))
    linkage_quality = len(matched_pairs) / len(real_df)

    return linkage_quality




In [86]:
linkage_quality3 = calculate_linkage_quality(df_anon, df_ext, 0.9)

print("Linkage quality: ", linkage_quality3)

Linkage quality:  0.0


In [84]:
similarity_columns = df_anon.columns.tolist()

# create similarity matrix
similarity_matrix = np.zeros((len(df_anon), len(df_ext)))
for i in range(len(df_anon)):
    for j in range(len(df_ext)):
        similarities = []
        for column in similarity_columns:
            if df_anon.at[i, column] == df_ext.at[j, column]:
                similarities.append(1)
            else:
                similarities.append(0)
        similarity_matrix[i, j] = sum(similarities) / len(similarity_columns)


In [85]:
similarity_matrix

array([[0.31034483, 0.31034483, 0.27586207, ..., 0.48275862, 0.55172414,
        0.37931034],
       [0.34482759, 0.44827586, 0.34482759, ..., 0.37931034, 0.48275862,
        0.51724138],
       [0.31034483, 0.31034483, 0.27586207, ..., 0.48275862, 0.55172414,
        0.37931034],
       ...,
       [0.27586207, 0.13793103, 0.48275862, ..., 0.37931034, 0.27586207,
        0.17241379],
       [0.37931034, 0.55172414, 0.34482759, ..., 0.44827586, 0.48275862,
        0.5862069 ],
       [0.20689655, 0.31034483, 0.4137931 , ..., 0.27586207, 0.27586207,
        0.34482759]])

In [102]:
from scipy.spatial.distance import cdist
similarity_columns = df_anon.columns.tolist()
anon_data = df_anon[similarity_columns].values
real_data = df_ext[similarity_columns].values
similarity_matrix =  ( cdist(anon_data, real_data, 'euclidean'))

ValueError: Unsupported dtype object

array([['Breast Invasive Ductal Carcinoma', 'Positive', 'Negative', ...,
        'BREAST CONSERVING', '1:Recurred', 117.80145386984807],
       ['Breast Invasive Ductal Carcinoma', 'Positive', 'Negative', ...,
        'MASTECTOMY', '0:Not Recurred', 110.31308011418916],
       ['Breast Invasive Ductal Carcinoma', 'Positive', 'Negative', ...,
        'BREAST CONSERVING', '1:Recurred', 120.44663442384136],
       ...,
       ['Breast Invasive Ductal Carcinoma', 'Negative', 'Negative', ...,
        'BREAST CONSERVING', '1:Recurred', 53.368576618432506],
       ['Breast Invasive Ductal Carcinoma', 'Positive', 'Negative', ...,
        'MASTECTOMY', '0:Not Recurred', 220.2934806092589],
       ['Breast Invasive Ductal Carcinoma', 'Negative', 'Negative', ...,
        'MASTECTOMY', '0:Not Recurred', 167.2542689522702]], dtype=object)